In [5]:
%load_ext autoreload
%autoreload 2

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision as tv
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()   # interactive mode

In [15]:
_include_('curriculum_vqa')
_include_('commons')

from cvqa import datasets, models, trainers
from commons import debug

seed = 1

np.random.seed(seed)
torch.manual_seed(seed);

root = f'{DEV_HOME}/curriculum_vqa/data-bin/basic_curriculum'

train_dataset = datasets.Curriculum(root, 'train')
train_dataset

Root: /Users/urisherman/Work/workspace/curriculum_vqa/data-bin/basic_curriculum/train 
Samples: 2000 (N_prompt=8, N_target=1)
Concepts: 0 
Classes: 0 
Vocab Tokens:19

In [24]:
B, N, d = 5, 3, 8

x = torch.rand(B, N, d)


F.pad(x, (0, 0, 0, 1), value=1)

tensor([[[4.5339e-01, 5.1638e-02, 2.9266e-02, 3.1533e-02, 9.8024e-01,
          8.9547e-01, 6.9834e-01, 3.7441e-01],
         [9.3050e-01, 3.6793e-01, 8.4711e-01, 4.8373e-01, 2.6480e-01,
          3.6045e-01, 3.9324e-01, 4.2332e-02],
         [6.4952e-01, 3.4766e-01, 7.2972e-01, 8.1879e-01, 4.2051e-01,
          7.4249e-01, 5.9990e-01, 8.3578e-01],
         [1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
          1.0000e+00, 1.0000e+00, 1.0000e+00]],

        [[1.7755e-01, 4.5610e-01, 3.3339e-01, 9.3812e-01, 5.1204e-02,
          2.1896e-01, 5.5645e-01, 9.4387e-01],
         [8.3831e-01, 3.2121e-02, 1.4847e-01, 9.8480e-01, 5.7287e-01,
          7.1747e-02, 3.4247e-01, 7.1859e-01],
         [3.9545e-01, 1.1594e-01, 1.9129e-01, 2.4098e-01, 4.8042e-01,
          6.9595e-01, 7.4006e-01, 2.5710e-01],
         [1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
          1.0000e+00, 1.0000e+00, 1.0000e+00]],

        [[4.1611e-01, 1.1396e-01, 8.7812e-01, 9.9898e-01, 9.